In [55]:
%pip install -e ..

Obtaining file:///home/rstudio/rl4greencrab
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for rl4greencrab (pyproject.toml) ... done
  Created wheel for rl4greencrab: filename=rl4greencrab-1.0.0-py2.py3-none-any.whl size=1073 sha256=b125839fccc41947614acd0fb477a4cd383c45dfc4c4d3f62424b7ba0fe2f36b
  Stored in directory: /tmp/pip-ephem-wheel-cache-jz0gv_n_/wheels/e9/7e/e6/00c4b11a2574abd59d64425d537139e25fadbde37f002c4dba
Successfully built rl4greencrab
  Attempting uninstall: rl4greencrab
    Found existing installation: rl4greencrab 1.0.0
    Uninstalling rl4greencrab-1.0.0:
      Successfully uninstalled rl4greencrab-1.0.0

[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the 

In [5]:
%pip install gym
%pip install optuna
%pip install rl_zoo3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached gymnasium-0.29.1-py3-none-any.whl.metadata (10 kB)
Using cached gymnasium-0.29.1-py3-none-any.whl (953 kB)
  Attempting uninstall: gymnasium
    Found existing installation: gymnasium 1.0.0a2
    Uninstalling gymnasium-1.0.0a2:
      Successfully uninstalled gymnasium-1.0.0a2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
shimmy 2.0.0 requires gymnasium>=1.0.0a1, but you have gymnasium 0.29.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [84]:
import numpy as np
import pandas as pd
from plotnine import ggplot, aes, geom_density, geom_line, geom_point, geom_violin, facet_grid, labs, theme, facet_wrap

from stable_baselines3 import PPO, TD3
from sb3_contrib import TQC
from stable_baselines3.common.env_util import make_vec_env

from rl4greencrab import simulator
import gym

import optuna
from optuna.pruners import MedianPruner
from optuna.samplers import TPESampler
from optuna.visualization import plot_optimization_history, plot_param_importances
from commented_env import greenCrabSimplifiedEnv
import gymnasium as gym
import logging
import numpy as np

from gymnasium import spaces
from scipy.stats import norm

import sample_params

In [85]:
logging.basicConfig(filename="/home/rstudio/rl4greencrab/notebooks/env_log.txt",format='%(levelname)s: %(message)s', level=logging.DEBUG)

class greenCrabEnv(gym.Env):
    
    metadata = {"render.modes": ["human"]}

    def __init__(
        self,
        config=None,
    ):
        # if config == {}:
        #     config = {
        #         "Tmax": 100,
        #         "growth_k": 0.43, "growth_xinf": 109, "growth_sd": 2.5, "nmortality": 0.03,
        #         "trapm_sigma": 0.15, "trapm_xmax": 44, "trapm_pmax": 0.0005, "trapf_pmax": 0.0008,
        #         "trapf_k": 0.5, "trapf_midpoint": 45, "init_mean_recruit": 15, "init_sd_recruit": 1.5,
        #         "init_mean_adult": 65, "init_sd_adult": 8, "init_n_recruit": 1000, "init_n_adult": 1000,
        #         "w_mort_scale": 5, "K": 25000, "imm": 10, "r": 50, "area": 4000,"loss_a": 0.265,
        #         "loss_b": 2.80, "loss_c": 2.99, "minsize": 5, "maxsize": 110, "nsize": 21, "ntime":9,"delta_t": 1/12,
        #         "env_stoch": 0.1, "action_reward_scale":0.001
        #     }
        
        config=config or {}
        
        # parameters
        self.growth_k = np.float32(config.get("growth_k", 0.43))
        self.growth_xinf = np.float32(config.get("growth_xinf", 109))
        self.growth_sd = np.float32(config.get("growth_sd", 2.5))
        self.nmortality = np.float32(config.get("nmortality", 0.03))
        
        self.trapm_sigma = np.float32(config.get("trapm_sigma", 6))
        self.trapm_xmax = np.float32(config.get("trapm_xmax", 47))
        self.trapm_pmax = np.float32(config.get("trapm_pmax", 2.26e-6))
        #
        self.trapf_pmax = np.float32(config.get("trapf_pmax", 8.3e-7))
        self.trapf_k = np.float32(config.get("trapf_k", 0.4))
        self.trapf_midpoint = np.float32(config.get("trapf_midpoint", 41))
        #
        self.traps_pmax = np.float32(config.get("traps_pmax", 2.75e-5))
        self.traps_k = np.float32(config.get("traps_k", 0.4))
        self.traps_midpoint = np.float32(config.get("traps_midpoint", 45))
        
        self.init_mean_recruit = config.get("init_mean_recruit", 15)
        self.init_sd_recruit = config.get("init_sd_recruit", 1.5)
        self.init_mean_adult = config.get("init_mean_adult", 65)
        self.init_sd_adult = config.get("init_sd_adult", 8)
        self.init_n_recruit = config.get("init_n_recruit", 0)
        self.init_n_adult = config.get("init_n_adult", 0)
        
        self.w_mort_scale = config.get("w_mort_scale", 5)
        self.K = config.get("K", 25000) #carrying capacity
        self.imm = config.get("imm", 1000) #colonization/immigration rate
        self.r = config.get("r", 1) #intrinsic rate of growth

        self.max_action = config.get("max_action", 2000)
        self.max_obs = config.get("max_obs", 2000)
        
        self.area = config.get("area", 4000)
        self.loss_a = config.get("loss_a", 0.265)
        self.loss_b = config.get("loss_b", 2.80)
        self.loss_c = config.get("loss_c", 2.99)
        
        self.minsize = config.get("minsize", 5)
        self.maxsize = config.get("maxsize", 110)
        self.nsize = config.get("nsize", 21)
        self.ntime = config.get("ntime", 9)
        
        self.delta_t = config.get("delta_t", 1/12)
        self.env_stoch = config.get("env_stoch", 0.1)
        
        self.action_reward_scale = np.array(config.get("action_reward_scale", [0.1, 0.1, 10]))
        self.action_reward_exponent = config.get("action_reward_exponent", 1)
        
        self.config = config

        # Preserve these for reset
        self.observations = np.zeros(shape=9, dtype=np.float32)
        self.reward = 0
        self.years_passed = 0
        self.Tmax = config.get("Tmax", 100)
                
        # Initial variables
        self.bndry = self.boundary()
        self.state = self.init_state()
        self.midpts = self.midpoints()
        self.gm_ker = self.g_m_kernel()
        self.w_mort = self.w_mortality()
        self.w_mort_exp = np.exp(-self.w_mort)
        self.pmort = np.exp(-self.nmortality)

        # Action space
        # action -- # traps per month
        self.action_space = spaces.Box(
            np.array([0, 0, 0], dtype=np.float32),
            np.array(3*[self.max_action], dtype=np.float32),
            dtype=np.float32,
        )
        
        # Observation space
        self.observation_space = spaces.Box(
            np.zeros(shape=self.ntime, dtype=np.float32),
            self.max_obs * np.ones(shape=self.ntime, dtype=np.float32),
            dtype=np.float32,
        )
        
    def step(self, action):
        # Size selective harvest rate, given action
        harvest_rate = (
            1 - np.exp(-(
                self.size_sel_norm() * action[0] +
                self.size_sel_log(self.trapf_pmax, self.trapf_midpoint, self.trapf_k) * action[1] +
                self.size_sel_log(self.traps_pmax, self.traps_midpoint, self.traps_k) * action[2]
            ))
        )
        
        # Add population at t=1
        size_freq = np.zeros(shape=(self.nsize, self.ntime), dtype='object')
        size_freq[:, 0] = self.state
        
        # Create array to store # removed
        removed = np.zeros(shape=(self.nsize, self.ntime), dtype='object')
        
        # Calculate removed and record observation at t=1
        removed[:, 0] = [np.random.binomial(size_freq[k, 0], harvest_rate[k]) for k in range(self.nsize)]
        self.observations[0] = np.sum(removed[:, 0])
        
        # Loop through intra-annual change (9 total months), t=2+
        for j in range(self.ntime - 1):
            n_j = self.gm_ker @ (size_freq[:, j] - removed[:, j])
            size_freq[:, j+1] = [np.random.binomial(n=n_j[k], p=self.pmort) for k in range(self.nsize)]
            removed[:, j+1] = [np.random.binomial(size_freq[k, j+1], harvest_rate[k]) for k in range(self.nsize)]
        
        # Record the catch in the observation space
        self.observations = np.array([np.sum(removed[:, j]) for j in range(self.ntime)], dtype=np.float32)
        
        # Calculate new adult population after overwinter mortality
        new_adults = [np.random.binomial(size_freq[k, 8], self.w_mort_exp[k]) for k in range(self.nsize)]
        
        # Simulate new recruits
        local_recruits = np.random.normal(self.dd_growth(size_freq[:, self.ntime-1]), self.env_stoch)
        nonlocal_recruits = np.random.poisson(self.imm) * (1 - np.sum(size_freq[:, self.ntime-1]) / self.K)
        recruit_total = local_recruits + nonlocal_recruits
        
        # Get sizes of recruits
        recruit_sizes = (norm.cdf(self.bndry[1:(self.nsize+1)], self.init_mean_recruit, self.init_sd_recruit) -
                         norm.cdf(self.bndry[0:self.nsize], self.init_mean_recruit, self.init_sd_recruit)) * recruit_total
        
        # Store new population size (and cap off at zero pop)
        self.state = np.maximum(recruit_sizes + new_adults, 0)
        
        # Calculate reward
        self.reward = self.reward_func(action)
        self.years_passed += 1
        
        done = bool(self.years_passed > self.Tmax)
        
        return self.observations, self.reward, done, done, {}


        
    def reset(self, *, seed=42, options=None):
        self.state = self.init_state()
        self.years_passed = 0

        # for tracking only
        self.reward = 0

        self.observations = np.zeros(shape=self.ntime, dtype=np.float32)

        return self.observations, {}

    #################
    #helper functions

    #set up boundary points of IPM mesh
    def boundary(self):
        boundary = self.minsize+np.arange(0,(self.nsize+1),1)*(self.maxsize-self.minsize)/self.nsize
        return boundary

    #set up mid points of IPM mesh
    def midpoints(self):
        midpoints = 0.5*(self.bndry[0:self.nsize]+self.bndry[1:(self.nsize+1)])
        return midpoints

    #function for initial state
    def init_state(self):
        init_pop = (norm.cdf(self.bndry[1:(self.nsize+1)],self.init_mean_adult,self.init_sd_adult)-\
         norm.cdf(self.bndry[0:self.nsize],self.init_mean_adult,self.init_sd_adult))*self.init_n_adult+\
        (norm.cdf(self.bndry[1:(self.nsize+1)],self.init_mean_recruit,self.init_sd_recruit)-\
         norm.cdf(self.bndry[0:self.nsize],self.init_mean_recruit,self.init_sd_recruit))*self.init_n_recruit
        return init_pop

    #function for logistic size selectivity curve
    def size_sel_log(self, trap_pmax, trap_midpts, trap_k):
        size_sel = trap_pmax/(1+np.exp(-trap_k*(self.midpts-trap_midpts)))
        return size_sel

    #function for gaussian size selectivity curve
    def size_sel_norm(self):
        size_sel = self.trapm_pmax*np.exp(-(self.midpts-self.trapm_xmax)**2/(2*self.trapm_sigma**2))
        return size_sel

    #function for growth/mortality kernel
    def g_m_kernel(self):
        array = np.empty(shape=(self.nsize,self.nsize),dtype='object')
        for i in range(self.nsize):
            mean = (self.growth_xinf-self.midpts[i])*(1-np.exp(-self.growth_k*self.delta_t)) + self.midpts[i]
            array[:,i] = (norm.cdf(self.bndry[1:(self.nsize+1)],mean,self.growth_sd)-\
                          norm.cdf(self.bndry[0:self.nsize],mean,self.growth_sd))
        return array

    #function for overwinter mortality
    def w_mortality(self):
        wmort = self.w_mort_scale/self.midpts
        return wmort

    #function for density dependent growth
    def dd_growth(self,popsize):
        dd_recruits = np.sum(popsize)*self.r*(1-np.sum(popsize)/self.K)
        return dd_recruits

    #function for reward
    # two part reward function:
    # 1. impact on environment (function of crab density)
    # 2. penalty for how much effort we expended (function of action)
    def reward_func(self,action):
        def trap_cost(action, max_action, exponent):
            return np.array(
                [
                    (action[0]/max_action) ** exponent,
                    (action[1]/max_action) ** exponent,
                    (action[2]/max_action) ** exponent,
                ]
            )
        reward = (
            -self.loss_a 
            /
            (
                1+np.exp(-self.loss_b*(np.sum(self.state)/self.area-self.loss_c))
            )
            - np.sum(
                self.action_reward_scale 
                * trap_cost(action, self.max_action, self.action_reward_exponent) 
            )
        )
        return reward


class greenCrabSimplifiedEnv(greenCrabEnv):
    """ like invasive_IPM but with simplified observations and normalized to -1, 1 space. """
    def __init__(self, config={}):
        super().__init__(config=config)
        self.observation_space = spaces.Box(
            np.array([-1,-1], dtype=np.float32),
            np.array([1,1], dtype=np.float32),
            dtype=np.float32,
        )
        self.action_space = spaces.Box(
            np.float32([-1, -1, -1]),
            np.float32([1, 1, 1]),
            dtype=np.float32,
        )
        self.max_action = config.get('max_action', 2000) # ad hoc based on previous values
        self.cpue_normalization = config.get('cpue_normalization', 100)
        
    def step(self, action):
        action_natural_units = np.maximum( self.max_action * (1 + action)/2 , 0.)
        obs, rew, term, trunc, info = super().step(
            np.float32(action_natural_units)
        )
        normalized_cpue = 2 * self.cpue_2(obs, action_natural_units) - 1
        # observation = np.float32(np.append(normalized_cpue, action))
        observation = normalized_cpue
        rew = 10 * rew # use larger rewards, possibly makes trainer easier?
        return observation, rew, term, trunc, info

    def reset(self, *, seed=42, options=None):
        _, info = super().reset(seed=seed, options=options)

        # completely new  obs
        return - np.ones(shape=self.observation_space.shape, dtype=np.float32), info

    def cpue_2(self, obs, action_natural_units):
        if any(action_natural_units <= 0):
            return np.float32([0,0])
        cpue_2 = np.float32([
            np.sum(obs[0:5]) / (self.cpue_normalization * action_natural_units[0]),
            np.sum(obs[5:]) / (self.cpue_normalization * action_natural_units[0])
        ])
        return cpue_2

In [86]:
gcse = greenCrabSimplifiedEnv()
vec_env = make_vec_env(greenCrabSimplifiedEnv, n_envs=12)
eval_envs = vec_env

In [88]:
model = PPO("MlpPolicy", vec_env, verbose=0, 
            batch_size =  64, # no
            n_steps = 32, # no
            gamma = 0.95, # no
            learning_rate =  0.9, # orginal = 0.9447561168646137
            ent_coef = 7.676877009214456e-07, # no
            clip_range =  0.1, # no
            n_epochs = 20, 
            gae_lambda = 0.9, # no
            max_grad_norm = 5, # no
            vf_coef = 0.9694540149524578, # no
            tensorboard_log="/home/rstudio/logs")
model.learn(
	total_timesteps=5, 
	progress_bar=True,
)
# model.save("ppo_gcse")

LiveError: Only one live display may be active at once

In [ ]:
hyperparameter = {'gamma': 0.9999, 
                  'learning_rate': 0.020439420278073966, 
                  'batch_size': 16, 'buffer_size': 10000, 
                  'learning_starts': 0, 
                  'train_freq': 16, 
                  'tau': 0.05, 
                  'log_std_init': -2.392885376919297, 
                  'net_arch': 'big', 
                  'n_quantiles': 31, 
                  'top_quantiles_to_drop_per_net': 1}

In [89]:
model = TQC("MlpPolicy", 
            vec_env, 
            verbose=0, 
            gamma= 0.9999, 
            learning_rate = 0.020439420278073966, 
            batch_size = 16, 
            buffer_size = 10000, 
            learning_starts= 0, 
            train_freq = 16, 
            tau = 0.05, 
            top_quantiles_to_drop_per_net = 1)
model.learn(
	total_timesteps=1000000, 
	progress_bar=True,
)

LiveError: Only one live display may be active at once

In [39]:
gcse.reward_func([0,0,0])

-6.126776689500633e-05

In [14]:
from stable_baselines3.common.evaluation import evaluate_policy

In [51]:
ppoAgent = PPO.load("ppo_gcse")
tunningAgent = TD3.load("tunning_best_gcse")
evalEnv = greenCrabSimplifiedEnv()

In [52]:
mean_rew, std_rew = evaluate_policy(tunningAgent, evalEnv)
print(f"TD3 reward = {mean_rew:.5f} +/- {std_rew:.5f}")

 action: [0.09852648 0.26327372 0.        ], reward: -0.00013980890837148188
 action: [0.09852648 0.26327372 0.        ], reward: -0.00032913614637149923
 action: [0.09852648 0.26327372 0.        ], reward: -0.0010596732821369717
 action: [0.09852648 0.26327372 0.        ], reward: -0.004963474836793878
 action: [0.09852648 0.26327372 0.        ], reward: -0.030974938258466016
 action: [0.09852648 0.26327372 0.        ], reward: -0.14113779899025197
 action: [0.09852648 0.26327372 0.        ], reward: -0.2376158711434138
 action: [0.09852648 0.26327372 0.        ], reward: -0.2592554098044239
 action: [0.09852648 0.26327372 0.        ], reward: -0.2630517205509694
 action: [0.09852648 0.26327372 0.        ], reward: -0.26391559677241017
 action: [0.09852648 0.26327372 0.        ], reward: -0.264230279692708
 action: [0.09852648 0.26327372 0.        ], reward: -0.2643721923266847
 action: [0.09852648 0.26327372 0.        ], reward: -0.26443932446594814
 action: [0.09852648 0.26327372 0.

In [7]:
model = PPO("MlpPolicy", vec_env, verbose=0, 
            learning_rate =  0.99, # orginal = 0.9447561168646137
            tensorboard_log="/home/rstudio/logs")
model.learn(
	total_timesteps=250_000, 
	progress_bar=True,
)
model.save("ppo_gcse")

Output()